MODELO DE RECOMENDACION

In [1]:
#primero importamos las librerias
import pandas as pd
import numpy as np
import ast
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#generamos un modelo de recomendacion en base a las etiquetas

games_steam = pd.read_csv('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/csv/games_steam_exportado_limpio.csv', encoding= 'utf-8')

In [3]:
#eliminamos los duplicados de la columna 'item_id'
games_steam.drop_duplicates(subset= 'item_id', inplace=True)

In [4]:
#vamos a extraer las columnas que necesitamos para nuestro modelo de recomendacion

games_steam = games_steam[['item_id', 'app_name', 'tags']]

In [5]:
#ahora visualizamos nuestro dataframe

games_steam

,item_id,app_name,tags
0,761140.0,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim..."
5,643980.0,Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '..."
9,670290.0,Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu..."
14,767400.0,弹炸人2222,"['Action', 'Adventure', 'Casual']"
17,773570.0,Log Challenge,"['Action', 'Indie', 'Casual', 'Sports']"
...,...,...,...
74821,773640.0,Colony On Mars,"['Strategy', 'Indie', 'Casual', 'Simulation']"
74825,733530.0,LOGistICAL: South Africa,"['Strategy', 'Indie', 'Casual']"
74828,610660.0,Russian Roads,"['Indie', 'Simulation', 'Racing']"
74831,658870.0,EXIT 2 - Directions,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ..."


In [6]:
#informacion general
games_steam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 0 to 74833
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   item_id   32132 non-null  float64
 1   app_name  32131 non-null  object 
 2   tags      31970 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1004.2+ KB


In [7]:
#eliminamos los nulos 
games_steam = games_steam.dropna()

In [8]:
#procedemos a cambiar el tipo de dato de la colunma 'item_id' ya  que es de tipo 'str'
#y la cambiamos a tipo 'int'
games_steam['item_id'] = games_steam['item_id'].astype(int)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_16756\1332479243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_steam['item_id'] = games_steam['item_id'].astype(int)


In [9]:
#cambiamos el tipo de dato de la columan 'tags' a lista
games_steam['tags'] = games_steam['tags'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_16756\705828889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_steam['tags'] = games_steam['tags'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])


In [10]:
#verificamos el paso anterior
type(games_steam['tags'][0])

list

In [11]:
#procedemos a separar las etiquetas
games = games_steam['tags'].apply(lambda x: ','.join(map(str,x)))
games = games.str.split(',', expand = True)

In [12]:
#rectificamos
games

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Strategy,Action,Indie,Casual,Simulation,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,Free to Play,Strategy,Indie,RPG,Card Game,Trading Card Game,Turn-Based,Fantasy,Tactical,Dark Fantasy,Board Game,PvP,2D,Competitive,Replay Value,Character Customization,Female Protagonist,Difficult,Design & Illustration,None
9,Free to Play,Simulation,Sports,Casual,Indie,Multiplayer,None,None,None,None,None,None,None,None,None,None,None,None,None,None
14,Action,Adventure,Casual,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
17,Action,Indie,Casual,Sports,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74821,Strategy,Indie,Casual,Simulation,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
74825,Strategy,Indie,Casual,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
74828,Indie,Simulation,Racing,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
74831,Indie,Casual,Puzzle,Singleplayer,Atmospheric,Relaxing,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [13]:
#generamos dummies
games = pd.get_dummies(games)

In [14]:
games

,0_2D,0_2D Fighter,0_360 Video,0_3D Platformer,0_4 Player Local,0_4X,0_Action,0_Action RPG,0_Adventure,0_America,...,19_Underwater,19_Utilities,19_VR,19_Video Production,19_Violent,19_Visual Novel,19_Voxel,19_Walking Simulator,19_Wargame,19_Zombies
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74821,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74825,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74828,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74831,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
#concatenamos los dataframes
games_steam_fin = pd.concat([games_steam, games], axis= 1)

In [16]:
games_steam_fin

,item_id,app_name,tags,0_2D,0_2D Fighter,0_360 Video,0_3D Platformer,0_4 Player Local,0_4X,0_Action,...,19_Underwater,19_Utilities,19_VR,19_Video Production,19_Violent,19_Visual Novel,19_Voxel,19_Walking Simulator,19_Wargame,19_Zombies
0,761140,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,643980,Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,670290,Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14,767400,弹炸人2222,"[Action, Adventure, Casual]",False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
17,773570,Log Challenge,"[Action, Indie, Casual, Sports]",False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74821,773640,Colony On Mars,"[Strategy, Indie, Casual, Simulation]",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74825,733530,LOGistICAL: South Africa,"[Strategy, Indie, Casual]",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74828,610660,Russian Roads,"[Indie, Simulation, Racing]",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74831,658870,EXIT 2 - Directions,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
#reindexamos
games_steam_fin.reset_index(drop= True, inplace=True)

In [18]:
#implementamos el coeficiente del coseno
similitud = cosine_similarity(games_steam_fin.iloc[:,3:])

In [19]:
similitud.shape

(31969, 31969)

In [20]:
#verificamos
similitud

array([[1.        , 0.10259784, 0.18257419, ..., 0.        , 0.        ,
        0.36514837],
       [0.10259784, 1.        , 0.09365858, ..., 0.        , 0.        ,
        0.09365858],
       [0.18257419, 0.09365858, 1.        , ..., 0.23570226, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.23570226, ..., 1.        , 0.23570226,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.23570226, 1.        ,
        0.        ],
       [0.36514837, 0.09365858, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [21]:
#dividimos el dataframe con la finalidad de deploy 

filas_cantidad = len(games_steam_fin)
filas_divididas = filas_cantidad // 10
modelo = games_steam_fin.iloc[:filas_divididas]

In [22]:
modelo.shape

(3196, 5457)

In [23]:
similitud_modelo = cosine_similarity(modelo.iloc[:,3:])

In [24]:
#exportamos
modelo.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/modelo.parquet')

In [25]:
#creamos la funcion
def recommend_games(id):
    game = modelo[modelo['item_id'].astype(int) == id]
    idx = game.index[0]

    sample_size = 2000
    sample_df = modelo.sample(n=sample_size, random_state=42)

    similitud = cosine_similarity([modelo.iloc[idx, 3:]], sample_df.iloc[:, 3:])
    similitud = similitud[0]

    juegmos_similares = [(i, similitud[i]) for i in range(len(similitud)) if i != idx]
    juegmos_similares = sorted(juegmos_similares, key=lambda x: x[1], reverse=True)

    juegmos_similares_ind = [i[0] for i in juegmos_similares[:5]]

    nombre_juegos_similares = sample_df['app_name'].iloc[juegmos_similares_ind].tolist()
    return {'juegos similares': nombre_juegos_similares}

In [26]:
#verificamos
recommend_games(767400)

{'juegos similares': ['Biozone',
  'Atomic Adam: Episode 1',
  'Scooby Doo! & Looney Tunes Cartoon Universe: Adventure',
  'Raid Mode Character: Albert Wesker',
  'Raid Mode: Album Storage B']}